# Setup

## Imports and fixed paths

These settings should work as they are without editing.

In [ ]:
# Imports

import os
import sys
import math
import random
import numpy as np
from netCDF4 import Dataset
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import xarray as xr
import pandas as pd
from importlib import reload 

In [ ]:
#Paths

# Getting the path of the notebook (NOTE: not totally safe)
localdir = os.getcwd()

# Adding the code directory to the path (one directory up from notebook directory and then src/)
codedir = os.path.join(os.path.dirname(localdir), 'src')

sys.path.append(codedir)
from track_loc import track_loc
from plot_trajectories import plot_traj

## User-defined parameters

Here, a .json file is specified with at least one dataset giving buoy name, lat, lon and "enddate" which is the reference position. If in addition "firstenddate" is defined, this dataset is a stationary mooring dataset. Each dataset, which can be composed of an array of ice stations or moorings, has a name.

The period states the maximum period over which the ice parcel should be tracked. Note that if the concentration falls below a given threshold (default 15%) for a consecutive number of days (defauly 3), then the trajectory is closed. If all trajectories in the array are closed, then the processing finishes (so note that the code does not often reach 100% processed when tracking).

Please see the python code for other parameters which can be set.

In [ ]:
# Trajectory parameters
json_file = 'sudarco_cruises.json'
json_data_2022mo = 'AO2022_moorings'
json_data_2022st = 'AO2022_ice_stations'
json_data_2023st = 'AO2023_ice_stations'
period = '5y'

# Defining and creating an output directory
outdir = os.path.join(localdir, 'output')
if not os.path.isdir(outdir):
    os.makedirs(outdir)

# Backtracking

## Ice Stations

Ice stations are any buoys or stations which are on an ice floe and move with the ice. Each ice station gives an individual trajectory.

Here, two SUDARCO cruise ice station datasets are tracked, each containing multiple ice stations. 

Please note, that the processing does not reach 100% is not an error - it means that the trajectories ended due to low ice concentration before the user-defined maximum tracking time was reached.

The output of this code is a NetCDF file per dataset.

### Trajectories

In [ ]:
# 2022
bt_2022st = track_loc(json_file=json_file, json_data=json_data_2022st, period=period, outname=outdir,
                      force=True, verbose=True)

In [ ]:
# 2023
bt_2023st = track_loc(json_file=json_file, json_data=json_data_2023st, period=period, outname=outdir,
              force=True, verbose=True)

### Plotting

This code plots each dataset individually and saves as a PNG file, in addition to showing in the notebook. 

The colmode parameter refers to how the trajectories are coloured:
timestep - a continuous colour range showing the time elapsed in the trajectory.
id - a single colour per ice station or mooring.
myi - whether the ice is likely first-year ice or multi-year ice (survived one summer melt season).
enddate - this is specific to moorings, and shows the reference date for each trajectory.

In [ ]:
# 2022 Ice Stations
plot_traj(bt_2022st, region='ease-nh', colmode='timestep', output=outdir)
plot_traj(bt_2022st, region='ease-nh', colmode='id', output=outdir)
plot_traj(bt_2022st, region='ease-nh', colmode='myi', output=outdir)


In [ ]:
# 2023 Ice Stations
plot_traj(bt_2023st, region='ease-nh', colmode='timestep', output=outdir)
plot_traj(bt_2023st, region='ease-nh', colmode='id', output=outdir)
plot_traj(bt_2023st, region='ease-nh', colmode='myi', output=outdir)


In [ ]:
# With a background
bgf = 'https://thredds.met.no/thredds/dodsC/osisaf/met.no/reprocessed/ice/conc_cra_files/2023/06/ice_conc_nh_ease2-250_icdr-v3p0_202306121200.nc'
os.makedirs(os.path.join(outdir, 'bg'), exist_ok=True)
plot_traj(bt_2023st, region='ease-nh', colmode='timestep', output=os.path.join(outdir, 'bg'), bg_file=bgf, bg_var='ice_conc')


## Moorings

Moorings are buoys which are fixed at a particular position and do not move with the ice. In this case, two dates are defined, the primary reference date, and a second date which defines a time period over which the ice trajectories are found to find the likely ice origins.

Here a SUDARCO 2022 cruise dataset of moorings is used, with two moorings in the array.

### Trajectories

In [ ]:
bt_2022mo = track_loc(json_file=json_file, json_data=json_data_2022mo, period=period, outname=outdir,
                      force=True, verbose=True)

### Plotting

The stride is set here to 1 week. Since the dataset is large, it takes time to plot a whole year of trajectories, and can be easier to interpret with fewer.

In [ ]:
# Plotting with a 1-month stride (takes a long time to plot every day!)
plot_traj(bt_2022mo, region='ease-nh', colmode='enddate', output=outdir, stride='7d')
plot_traj(bt_2022mo, region='ease-nh', colmode='timestep', output=outdir, stride='7d')
plot_traj(bt_2022mo, region='ease-nh', colmode='id', output=outdir, stride='7d')
plot_traj(bt_2022mo, region='ease-nh', colmode='myi', output=outdir, stride='7d')


# Forward tracking

In addition to tracking backwards in time to where the ice likely first froze, it is possible to track forwards in time to where it melts again. This example, also using the SUDARCO 2022 ice stations, shows these ice floes moving down the Fram Strait where they melt.

## Ice Stations

### Trajectories

In [ ]:
# 2022
ft_2022st = track_loc(json_file=json_file, json_data=json_data_2022st, period=period, outname=outdir,
                      forwardtrack=True, force=True, verbose=True)

### Plotting

In [ ]:
# 2022 Ice Stations
plot_traj(ft_2022st, region='ease-nh-wide', colmode='timestep', output=outdir)